
# EMA(20/50/200) + ADX(14) Screener **AS-OF Date** (NSE)

Generate a **historical** screener CSV **as of a given date** (IST), not just "today".  
This lets you create point-in-time signal files for your backtest.

**What changes vs the regular screener?**
- New parameter: `ASOF_DATE_STR = "YYYY-MM-DD"` (IST).  
- Data are sliced **up to and including** this date. No future bars leak into signals.
- Output file name is suffixed with the as-of date you choose.

**Outputs**
- `outputs/swing_ema_adx_candidates_<ASOF_DATE>.csv`

> Notes
> - Indicators computed only on historical bars up to `ASOF_DATE`.
> - Use with the companion backtest notebook (entries on next day open).



## 0) Environment
If needed, install dependencies (uncomment to run once):
```python
# %pip install yfinance pandas pandas_ta numpy pytz
```


In [1]:

import os, warnings
from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
from zoneinfo import ZoneInfo

warnings.filterwarnings("ignore")
pd.set_option("display.width", 140)
pd.set_option("display.max_rows", 200)

# ==============================
# USER PARAMETERS
# ==============================
TICKERS = ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AFCONS.NS', 'AFFLE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'ALIVUS.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GNFC.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GPPL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JUSTDIAL.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KANSAINER.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KNRCON.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MASTEK.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PEL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RAYMOND.NS', 'RAYMONDLSL.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RENUKA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'ROUTE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RTNINDIA.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SWSOLAR.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TANLA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WESTLIFE.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS']


INTERVAL = "1d"
LOOKBACK_YEARS = 5

EMA_FAST = 20
EMA_SLOW = 50
EMA_LONG = 200
ADX_LEN  = 14
ADX_THRESHOLD = 25.0

# Accept signals that happened within last N bars **as of AS-OF DATE**.
RECENT_BARS = 0   # 0 = only on the last bar as of AS-OF DATE

# ---- NEW: AS-OF DATE (IST) ----
ASOF_DATE_STR = "2024-12-31"  # <-- set to any YYYY-MM-DD you want (IST)
# -------------------------------

OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

IST = ZoneInfo("Asia/Kolkata")
ASOF_DATE = datetime.strptime(ASOF_DATE_STR, "%Y-%m-%d").replace(tzinfo=IST)



## 1) Helpers
- `fetch_ohlcv_asof`: fetches OHLCV ending at your as-of date (inclusive) and localizes to IST.
- `compute_indicators` and `generate_signals`: same rules as the live screener.


In [2]:

def fetch_ohlcv_asof(ticker: str, years: int, interval: str, asof_dt) -> pd.DataFrame:
    # Start enough history; end set just after AS-OF to ensure inclusion
    start = (asof_dt - pd.Timedelta(days=365*years)).date().isoformat()
    end   = (asof_dt + pd.Timedelta(days=1)).date().isoformat()
    df = yf.download(ticker, start=start, end=end, interval=interval, auto_adjust=False, progress=False, multi_level_index=False)
    if df.empty:
        return df
    # yfinance daily is naive -> localize to UTC then convert to IST
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC").tz_convert(IST)
    else:
        df.index = df.index.tz_convert(IST)
    df = df.rename(columns=str.title)
    # Strictly cut off any rows beyond AS-OF date (defensive)
    df = df[df.index <= asof_dt]
    return df

def compute_indicators(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    out = df.copy()
    out[f"EMA{EMA_FAST}"] = ta.ema(out["Close"], length=EMA_FAST)
    out[f"EMA{EMA_SLOW}"] = ta.ema(out["Close"], length=EMA_SLOW)
    out[f"EMA{EMA_LONG}"] = ta.ema(out["Close"], length=EMA_LONG)
    adx = ta.adx(out["High"], out["Low"], out["Close"], length=ADX_LEN)
    if adx is not None and not adx.empty:
        out["ADX"] = adx[f"ADX_{ADX_LEN}"]
        out["+DI"] = adx[f"DMP_{ADX_LEN}"]
        out["-DI"] = adx[f"DMN_{ADX_LEN}"]
    else:
        out["ADX"] = np.nan; out["+DI"] = np.nan; out["-DI"] = np.nan
    return out

def generate_signals(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    out = df.copy()
    ef, es, el = f"EMA{EMA_FAST}", f"EMA{EMA_SLOW}", f"EMA{EMA_LONG}"
    out["bull_cross"] = (out[ef] > out[es]) & (out[ef].shift(1) <= out[es].shift(1))
    out["bear_cross"] = (out[ef] < out[es]) & (out[ef].shift(1) >= out[es].shift(1))

    uptrend   = out["Close"] > out[el]
    downtrend = out["Close"] < out[el]
    strong    = out["ADX"] >= ADX_THRESHOLD

    out["LongSignal"]  = uptrend & (out[ef] > out[es]) & out["bull_cross"] & strong
    out["ShortSignal"] = downtrend & (out[ef] < out[es]) & out["bear_cross"] & strong
    return out



## 2) Run Screener **as of** the given date
Only bars **≤ AS-OF DATE** are considered for signal detection.


In [3]:

rows = []
for tk in TICKERS:
    try:
        df = fetch_ohlcv_asof(tk, LOOKBACK_YEARS, INTERVAL, ASOF_DATE)
        if df.empty or len(df) < max(EMA_LONG, ADX_LEN) + 5:
            continue
        df = compute_indicators(df)
        df = generate_signals(df).dropna(subset=[f"EMA{EMA_LONG}","ADX"])
        if df.empty:
            continue

        # Focus strictly up to AS-OF last bar
        last_idx = df.index[-1]

        if RECENT_BARS <= 0:
            last = df.iloc[-1]
            if last["LongSignal"] or last["ShortSignal"]:
                rows.append({
                    "Ticker": tk,
                    "Date": last_idx.date().isoformat(),
                    "Signal": "LONG" if last["LongSignal"] else "SHORT",
                    "Close": round(float(last["Close"]), 2),
                    f"EMA{EMA_FAST}": round(float(last[f"EMA{EMA_FAST}"]), 2),
                    f"EMA{EMA_SLOW}": round(float(last[f"EMA{EMA_SLOW}"]), 2),
                    f"EMA{EMA_LONG}": round(float(last[f"EMA{EMA_LONG}"]), 2),
                    "ADX": round(float(last["ADX"]), 2),
                })
        else:
            window = df.iloc[-RECENT_BARS:]
            hits = window[(window["LongSignal"]) | (window["ShortSignal"])]
            for idx, row in hits.iterrows():
                rows.append({
                    "Ticker": tk,
                    "Date": idx.date().isoformat(),
                    "Signal": "LONG" if row["LongSignal"] else "SHORT",
                    "Close": round(float(row["Close"]), 2),
                    f"EMA{EMA_FAST}": round(float(row[f"EMA{EMA_FAST}"]), 2),
                    f"EMA{EMA_SLOW}": round(float(row[f"EMA{EMA_SLOW}"]), 2),
                    f"EMA{EMA_LONG}": round(float(row[f"EMA{EMA_LONG}"]), 2),
                    "ADX": round(float(row["ADX"]), 2),
                })
    except Exception as e:
        print(f"[WARN] {tk}: {e}")

res_df = pd.DataFrame(rows).sort_values(["Signal","ADX"], ascending=[True, False]).reset_index(drop=True)
res_df



1 Failed download:
['ALIVUS.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-02 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1577903400, endDate = 1735669800")')

1 Failed download:
['COHANCE.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-02 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1577903400, endDate = 1735669800")')

1 Failed download:
['ETERNAL.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-02 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1577903400, endDate = 1735669800")')


,Ticker,Date,Signal,Close,EMA20,EMA50,EMA200,ADX
0,LTIM.NS,2024-12-30,SHORT,5643.50,6087.63,6112.65,5771.53,32.00
1,TTML.NS,2024-12-30,SHORT,74.49,78.30,78.44,82.06,26.96



## 3) Save CSV (dated by your AS-OF)


In [4]:

out_path = os.path.join(OUT_DIR, f"swing_ema_adx_candidates_{ASOF_DATE.date().isoformat()}.csv")
if res_df.empty:
    print(f"No signals as of {ASOF_DATE.date().isoformat()} for the configured parameters.")
else:
    res_df.to_csv(out_path, index=False)
    print(f"Saved {len(res_df)} candidates -> {out_path}")


Saved 2 candidates -> outputs/swing_ema_adx_candidates_2024-12-31.csv



## 4) Tip for backtesting
- Set the backtest notebook's `INPUT_CSV` to this file (same date).
- The backtest will simulate entry on the **next trading day open** after each `Date` in the CSV.
- Keep the indicator settings here equal to your backtest settings for consistency.
